In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

import pandas as pd
import numpy as np

In [20]:
# data built in /retention\ data\ build.ipynb
org_year = pd.read_csv("org_year.csv")
print("Max year: {}".format(org_year['year'].max()))
org_year.tail(3)

Max year: 2024


,org,year,donors_unique,donors_retained,new_donors,recurring_donors,retention
26890,450221,2024,3,0,3,0,0.0
26891,451233,2024,1,0,1,0,0.0
26892,448174,2023,1,1,1,0,1.0


In [13]:
# load org segment
q = "select id as org, segment from organization"
org_ntee = redshift_query_read(q, schema='production')
org_ntee.head(2)

,org,segment
0,23,None
1,24,None


In [9]:
# load org segment
org_export = get_dataframe_from_file('qgiv-stats-data', 'org_download.csv')
org_export = org_export[['Id', 'Segment', 'Pricing Package']]
org_export.columns = ['org', 'segment', 'pricing_package']
org_export.head(3)

,org,segment,pricing_package
0,441789,Z - Unknown,Start
1,443044,NaN,Start
2,449429,NaN,Package Pricing


In [21]:
org_year = org_year.merge(org_ntee, on='org')

In [22]:
org_year.groupby('segment')['retention'].agg(['mean', 'median', 'count'])

,mean,median,count
segment,,,
A - Arts; Culture; and Humanities,0.306859,0.231451,1643
B - Educational Institutions,0.310567,0.254545,3185
C - Environmental Advocacy and Protection,0.319370,0.250000,753
D - Animal,0.333431,0.240221,802
E - Health; General and Rehabilitative,0.255170,0.200000,1346
F - Mental Health and Crisis Intervention,0.283755,0.208584,734
G - Disease; Disorders; Medical Disciplines,0.251054,0.190083,927
H - Medical Research,0.247445,0.180431,181
I - Crime; Legal Related,0.268969,0.213793,393


In [24]:
org_year.groupby(['segment', 'year'])[['retention', 'new_donors', 'recurring_donors']].agg(['mean', 'median', 'count'])

retention                  new_donors  \
                                             mean    median count        mean   
segment                            year                                         
A - Arts; Culture; and Humanities  2008  0.083179  0.083179     2  189.500000   
                                   2009  0.103998  0.090402     8  143.625000   
                                   2010  0.130263  0.166667    15  146.200000   
                                   2011  0.173522  0.151795    22  124.363636   
                                   2012  0.158318  0.150000    28  169.714286   
...                                           ...       ...   ...         ...   
Z - Unknown                        2020  0.359371  0.260172    48  164.354167   
                                   2021  0.418396  0.354167    40   76.975000   
                                   2022  0.324435  0.279188    41   83.682927   
                                   2023  0.450334  0.387387    33   72.181818   
                                   2024  0.453847  0.500000    27   37.481481   

                                                     recurring_donors         \
                                        median count             mean median   
segment                            year                                        
A - Arts; Culture; and Humanities  2008  189.5     2        16.000000   16.0   
                                   2009   62.5     8         8.625000    4.0   
                                   2010   71.0    15         8.066667    2.0   
                                   2011   50.5    22         8.954545    1.0   
                                   2012   69.0    28         9.714286    2.0   
...                                        ...   ...              ...    ...   
Z - Unknown                        2020   26.0    48         6.208333    0.0   
                                   2021    9.5    40         4.925000    0.0   
                                   2022   22.0    41         2.902439    0.0   
                                   2023   32.0    33         2.848485    0.0   
                                   2024   10.0    27         1.851852    0.0   

                                               
                                        count  
segment                            year        
A - Arts; Culture; and Humanities  2008     2  
                                   2009     8  
                                   2010    15  
                                   2011    22  
                                   2012    28  
...                                       ...  
Z - Unknown                        2020    48  
                                   2021    40  
                                   2022    41  
                                   2023    33  
                                   2024    27  

[418 rows x 9 columns]

In [31]:
agg = org_year.groupby(['segment', 'year'])[['retention', 'new_donors', 'recurring_donors']].agg(['mean', 'median', 'count']).reset_index()
#agg.pivot(index='segment', columns='year').to_csv("agg.csv", index=False)
agg.pivot(index='segment', columns='year').reset_index()

segment retention            \
                                                             mean             
year                                                         2007      2008   
0                    A - Arts; Culture; and Humanities        NaN  0.083179   
1                         B - Educational Institutions        NaN  0.031746   
2             C - Environmental Advocacy and Protection       NaN       NaN   
3                                           D - Animal        NaN       NaN   
4               E - Health; General and Rehabilitative        NaN  0.082258   
5            F - Mental Health and Crisis Intervention        NaN       NaN   
6          G - Disease; Disorders; Medical Disciplines        NaN  0.500000   
7                                 H - Medical Research        NaN       NaN   
8                             I - Crime; Legal Related        NaN       NaN   
9                          J - Employment; Job Related        NaN       NaN   
10                     K - Agriculture; Food; Nutrition       NaN       NaN   
11                                L - Housing; Shelter        NaN       NaN   
12                  M - Public Safety; Disaster Relief        NaN       NaN   
13          N - Recreation; Sports; Leisure; Athletics        NaN       NaN   
14                               O - Youth Development   0.166667  0.039679   
15                                  P - Human Services   0.066667  0.118056   
16    Q - International; Foreign Affairs; and Nation...       NaN       NaN   
17           R - Civil Rights; Social Action; Advocacy        NaN       NaN   
18    R - Civil Rights; Social Action; Advocacy ; Z ...       NaN       NaN   
19        S - Community Improvement; Capacity Building        NaN       NaN   
20      T - Philanthropy; Voluntarism; and Grantmaking        NaN  0.024181   
21    T - Philanthropy; Voluntarism; and Grantmaking...       NaN       NaN   
22       U - Science and Technology Research Institutes       NaN       NaN   
23              V - Social Science Research Institutes        NaN       NaN   
24                         W - Public; Society Benefit        NaN  0.181818   
25                 X - Religion; Spiritual Development        NaN  0.266791   
26          Y - Mutual/Membership Benefit Organizations       NaN       NaN   
27                                         Z - Unknown        NaN  0.416667   

                                                                            \
                                                                             
year      2009      2010      2011      2012      2013      2014      2015   
0     0.103998  0.130263  0.173522  0.158318  0.204105  0.245813  0.224704   
1     0.121040  0.206087  0.175996  0.208277  0.242484  0.244705  0.270637   
2     0.103622  0.221667  0.207531  0.215787  0.179524  0.239087  0.265384   
3          NaN  0.204588  0.145836  0.239840  0.221907  0.247745  0.365764   
4     0.175246  0.137177  0.190155  0.151790  0.185030  0.142304  0.167038   
5     0.157312  0.101283  0.148372  0.143946  0.156116  0.188796  0.238690   
6     0.102258  0.218111  0.173804  0.169248  0.168288  0.184915  0.230670   
7          NaN  0.045732  0.204755  0.133923  0.118646  0.196353  0.181931   
8     0.247826  0.142810  0.130442  0.126689  0.196744  0.202739  0.190597   
9          NaN       NaN  0.194444  0.041284  0.146667  0.422824  0.297601   
10    0.142857  0.025424  0.118615  0.163556  0.167397  0.201658  0.317393   
11    0.208955  0.115484  0.110761  0.125871  0.143627  0.187252  0.197692   
12         NaN       NaN       NaN  0.153846  0.125790  0.223764  0.137481   
13         NaN  0.171339  0.129548  0.138169  0.270808  0.208992  0.190126   
14    0.131659  0.198428  0.143503  0.182980  0.238792  0.246074  0.217312   
15    0.103725  0.181669  0.157444  0.227877  0.233848  0.234697  0.273022   
16    0.128079  0.134072  0.276441  0.276559  0.319623  0.354812  0.420924   
17         NaN  0.117343  0.186881  0.

__Civil rights, Unknown__ has the highest retention, I'm assuming because of the small sample size (2 organizations). __Religious__ is noteworthy as it has the second highest retention and the greatest sample size. __Medical research__, __disease__, and __health__ all have the lowest retention which is consipcous given the overlap in these categories and meaningful sample sizes.

In [13]:
org_year[org_year['year']>=2012].groupby(['year', 'segment'])['retention'].agg(['mean', 'median']).reset_index().pivot(index='segment', columns='year', values=['median'])

median            \
year                                                    2012      2013   
segment                                                                  
A - Arts, Culture, and Humanities                   0.150000  0.156288   
B - Educational Institutions                        0.161905  0.219298   
C - Environmental Advocacy and Protection           0.202797  0.164452   
D - Animal                                          0.172414  0.228916   
E - Health, General and Rehabilitative              0.127596  0.149816   
F - Mental Health and Crisis Intervention           0.101402  0.158415   
G - Disease, Disorders, Medical Disciplines         0.117381  0.141604   
H - Medical Research                                0.110092  0.103659   
I - Crime, Legal Related                            0.144538  0.164286   
J - Employment, Job Related                         0.041284  0.200000   
K - Agriculture, Food, Nutrition                    0.236842  0.219309   
L - Housing, Shelter                                0.134328  0.127273   
M - Public Safety, Disaster Relief                  0.153846  0.125790   
N - Recreation, Sports, Leisure, Athletics          0.119403  0.178571   
O - Youth Development                               0.144330  0.196646   
P - Human Services                                  0.205518  0.222810   
Q - International, Foreign Affairs, and Nationa...  0.238095  0.245732   
R - Civil Rights, Social Action, Advocacy           0.252381  0.270588   
R - Civil Rights, Social Action, Advocacy , Z -...       NaN       NaN   
S - Community Improvement, Capacity Building        0.281667  0.217647   
T - Philanthropy, Voluntarism, and Grantmaking      0.167785  0.217213   
U - Science and Technology Research Institutes           NaN  0.063866   
V - Social Science Research Institutes              0.454545  0.431723   
W - Public, Society Benefit                         0.115896  0.143843   
X - Religion, Spiritual Development                 0.312326  0.333333   
Y - Mutual/Membership Benefit Organizations              NaN  0.153846   
Z - Unknown                                         0.187500  0.251147   
Z - Unknown , T - Philanthropy, Voluntarism, an...       NaN       NaN   

                                                                        \
year                                                    2014      2015   
segment                                                                  
A - Arts, Culture, and Humanities                   0.195699  0.180291   
B - Educational Institutions                        0.209302  0.227596   
C - Environmental Advocacy and Protection           0.181818  0.219601   
D - Animal                                          0.217391  0.263158   
E - Health, General and Rehabilitative              0.144659  0.148148   
F - Mental Health and Crisis Intervention           0.196413  0.165843   
G - Disease, Disorders, Medical Disciplines         0.115780  0.132346   
H - Medical Research                                0.168865  0.138336   
I - Crime, Legal Related                            0.154762  0.135135   
J - Employment, Job Related                         0.450000  0.244718   
K - Agriculture, Food, Nutrition                    0.209546  0.308940   
L - Housing, Shelter                                0.167513  0.159806   
M - Public Safety, Disaster Relief                  0.293515  0.106250   
N - Recreation, Sports, Leisure, Athletics          0.160000  0.195238   
O - Youth Development                               0.224698  0.171742   
P - Human Services                                  0.212406  0.242000   
Q - International, Foreign Affairs, and Nationa...  0.310795  0.334175   
R - Civil Rights, Social Action, Advocacy           0.311111  0.259259   
R - Civil Rights, Social Action, Advocacy , Z -...       NaN       NaN   
S - Community Improvement, Capacity Building        0.260870  0.243050   
T - Philanthropy, Voluntarism, and Grantmaking      0.192

It is reassuring to see that in most categories retention is generally rising slowly over time.